In [26]:
"""
Travelling Salesman Problem Solver

This script implements a heuristic algorithm for the Travelling Salesman Problem (TSP) that uses the 2-opt local search to optimize an initial solution generated by the Randomized Nearest Neighbor algorithm. The TSP seeks to find the shortest route that visits every city in a given list exactly once and returns to the starting city.

This script requires Python 3.6 or later and depends on the `math` module.

Usage:
    Call the `tsp_solver` function with a list of cities and their coordinates, a distance matrix that stores the distance between every pair of cities, the number of iterations to run the algorithm, and the unit of measurement for the distance traveled. The function returns the best route found and its total distance in the specified unit of measurement.

Example:
    cities = [(0,0), (1,1), (2,2), (3,3), (4,4), (5,5)]
    dist_matrix = [[0, 1, 2, 3, 4, 5],
                   [1, 0, 1, 2, 3, 4],
                   [2, 1, 0, 1, 2, 3],
                   [3, 2, 1, 0, 1, 2],
                   [4, 3, 2, 1, 0, 1],
                   [5, 4, 3, 2, 1, 0]]
    iterations = 1000
    unit = 'kilometers'
    best_route, total_distance = tsp_solver(cities, dist_matrix, iterations, unit)
    print(f"Best route: {best_route}\nTotal distance: {total_distance} {unit}")

++++++++++++++++++++ FUNCTIONS TO BE IMPLEMENTED START ++++++++++++++++++++

Function: tsp_solver

    Solves the TSP for a given list of cities and their coordinates, distance matrix, number of iterations, and unit of
    measurement. Returns the best route and its total distance in the specified unit of measurement.

    Parameters:
        cities (list of tuples): A list of cities with their coordinates.
        dist_matrix (2D list): A distance matrix that stores the distance between every pair of cities.
        iterations (int): The number of iterations to run the algorithm.
        unit (str): The unit of measurement for the distance traveled (e.g. 'kilometers', 'miles', 'meters').

    Returns:
        tuple: A tuple containing the best route found and its total distance in the specified unit of measurement.

Function: randomized_nearest_neighbor

    Generates an initial solution to the TSP using the randomized nearest neighbor algorithm.

    Parameters:
        cities (list of tuples): A list of cities with their coordinates.
        dist_matrix (2D list): A distance matrix that stores the distance between every pair of cities.

                    #Distance Matrix Example
                    City A  City B  City C  City D
            City A       0     10      15      20
            City B      10      0      25      30
            City C      15     25       0      35
            City D      20     30      35       0

    Returns:
        list: A list representing the initial route generated by the algorithm.
    
Function: calculate_total_distance

    Calculates the total distance of a route given its distance matrix.

    Parameters:
        route (list): A list representing a route.
        dist_matrix (2D list): A distance matrix that stores the distance between every pair of cities.

    Returns:
        float: The total distance of the route.
    
Function: swap_cities

    Swaps two cities in a route.

    Parameters:
        route (list): A list representing a route.
        index1 (int): The index of the first city to swap.
        index2 (int): The index of the second city to swap.

    Returns:
        list: A list representing the new route generated by the swap.
    
Function: apply_2opt_swap

    Applies the 2-opt swap to a route.

    Parameters:
        route (list): A list representing a route.
        index1 (int): The index of the first city to swap.
        index2 (int): The index of the second city to swap.

    Returns:
        list: A list representing the new route generated by the 2-opt swap.

Function: tsp_algorithm

    Runs the TSP algorithm for a given number of iterations.

    Parameters:
        route (list): A list representing a route.
        dist_matrix (2D list): A distance matrix that stores the distance between every pair of cities.
        iterations (int): The number of iterations to run the algorithm.

    Returns:
        tuple: A tuple containing the best route found and its total distance.
    
Function: tsp_solver

    Solves the TSP for a given list of cities and their coordinates, distance matrix, number of iterations, and unit of
    measurement. Returns the best route and its total distance in the specified unit of measurement.

    Parameters:
        cities (list of tuples): A list of cities with their coordinates.
        dist_matrix (2D list): A distance matrix that stores the distance between every pair of cities.
        iterations (int): The number of iterations to run the algorithm.
        unit (str): The unit of measurement for the distance traveled (e.g. 'kilometers', 'miles', 'meters').

    Returns:
        tuple: A tuple containing the best route found and its total distance in the specified unit of measurement.

++++++++++++++++++++ FUNCTIONS TO BE IMPLEMENTED END ++++++++++++++++++++

Author: Carl Kho
Date created: March 8, 2023
Last modified: March 8, 2023
Python Version: 3.6 or later
"""

'\nTravelling Salesman Problem Solver\n\nThis script implements a heuristic algorithm for the Travelling Salesman Problem (TSP) that uses the 2-opt local search to optimize an initial solution generated by the Randomized Nearest Neighbor algorithm. The TSP seeks to find the shortest route that visits every city in a given list exactly once and returns to the starting city.\n\nThis script requires Python 3.6 or later and depends on the `math` module.\n\nUsage:\n    Call the `tsp_solver` function with a list of cities and their coordinates, a distance matrix that stores the distance between every pair of cities, the number of iterations to run the algorithm, and the unit of measurement for the distance traveled. The function returns the best route found and its total distance in the specified unit of measurement.\n\nExample:\n    cities = [(0,0), (1,1), (2,2), (3,3), (4,4), (5,5)]\n    dist_matrix = [[0, 1, 2, 3, 4, 5],\n                   [1, 0, 1, 2, 3, 4],\n                   [2, 1, 0

In [51]:
import random
import math


def swap_cities(best_route):
    # Randomly select two cities in the route and swap their positions
    new_route = best_route.copy()
    city1, city2 = random.sample(range(len(new_route)), 2)
    new_route[city1], new_route[city2] = new_route[city2], new_route[city1]
    return new_route


def two_opt_swap(new_route, dist_matrix):
    # Apply the 2-opt swap to the route to eliminate any crossovers or intersections
    for i in range(len(new_route) - 1):
        for j in range(i + 1, len(new_route)):
            if j - i == 1:
                continue  # changes nothing, skip then
            # reverse the path between i and j
            reversed_path = list(reversed(new_route[i:j]))
            # check if this reverse will cause improvement in distance
            if (dist_matrix[new_route[i - 1]][new_route[j]] + dist_matrix[new_route[i]][new_route[j + 1]]) > (
                    dist_matrix[new_route[i - 1]][new_route[i]] + dist_matrix[new_route[j]][new_route[j + 1]]):
                # apply the reversal
                new_route[i:j] = reversed_path
    return new_route


def convert_distance_unit(best_distance, distance_unit):
    # Convert the best distance to the specified unit of measurement
    if distance_unit == 'km':
        return round(best_distance, 2)
    elif distance_unit == 'm':
        return round(best_distance * 1000, 2)
    else:
        raise ValueError("Invalid distance unit. Choose 'km' or 'm'.")


def tsp_solver(city_coords, iterations, distance_unit):
    # Generate the distance matrix
    dist_matrix = generate_distance_matrix(city_coords)

    # Generate initial route using Randomized Nearest Neighbor algorithm
    initial_route = generate_initial_route(city_coords)
    print(f"Initial route: {initial_route}")
    initial_distance = calculate_total_distance(initial_route, dist_matrix)
    print(f"Initial distance: {initial_distance}{distance_unit}")

    # Set best_route to initial route and calculate its total distance
    best_route = initial_route
    best_distance = initial_distance
    print(f"Best route: {best_route}")
    print(f"Best distance: {best_distance}{distance_unit}")

    # Run the 2-opt algorithm for the specified number of iterations
    for i in range(iterations):
        # Randomly select two cities in the route and swap their positions
        new_route = swap_cities(best_route)
        print(f"Iteration {i+1}:")
        print(f"New route: {new_route}")

        # Apply the 2-opt swap to the route to eliminate any crossovers or intersections
        new_route = two_opt_swap(new_route, dist_matrix)
        print(f"2-opt swapped route: {new_route}")

        # Calculate the total distance of the new route
        new_distance = calculate_total_distance(new_route, dist_matrix)
        print(f"New distance: {new_distance}{distance_unit}")

        # If the new route is shorter than the previous best route, update best_route and its total distance
        if new_distance < best_distance:
            best_route = new_route
            best_distance = new_distance
            print(f"New best route: {best_route}")
            print(f"New best distance: {best_distance}{distance_unit}")
        else:
            print(f"No improvement in route for iteration {i+1}")

    # Convert the best distance to the specified unit of measurement
    best_distance = convert_distance_unit(best_distance, distance_unit)

    return best_route, best_distance


def swap_cities(best_route):
    # Randomly select two cities in the route and swap their positions
    new_route = best_route.copy()
    city1, city2 = random.sample(range(len(new_route)), 2)
    new_route[city1], new_route[city2] = new_route[city2], new_route[city1]
    return new_route


def two_opt_swap(new_route, dist_matrix):
    # Apply the 2-opt swap to the route to eliminate any crossovers or intersections
    for i in range(len(new_route) - 1):
        for j in range(i + 1, len(new_route)):
            if j - i == 1:
                continue  # changes nothing, skip then
            # reverse the path between i and j
            reversed_path = list(reversed(new_route[i:j]))
            # check if this reverse will cause improvement in distance
            if (dist_matrix[new_route[i - 1]][new_route[j]] + dist_matrix[new_route[i]][new_route[j + 1]]) > (
                    dist_matrix[new_route[i - 1]][new_route[i]] + dist_matrix[new_route[j]][new_route[j + 1]]):
                # apply the reversal
                new_route[i:j] = reversed_path
    return new_route


def convert_distance_unit(best_distance, distance_unit):
    # Convert the best distance to the specified unit of measurement
    if distance_unit == 'km':
        return round(best_distance, 2)
    elif distance_unit == 'm':
        return round(best_distance * 1000, 2)
    else:
        raise ValueError("Invalid distance unit. Choose 'km' or 'm'.")


def generate_distance_matrix(city_coords):
    dist_matrix = []
    for i, coord1 in enumerate(city_coords):
        row = []
        for j, coord2 in enumerate(city_coords):
            if i == j:
                row.append(0)
            else:
                dist = distance(coord1, coord2)
                row.append(dist)
        dist_matrix.append(row)
    return dist_matrix


def distance(coord1, coord2):
    # Calculate the Euclidean distance between two coordinates
    x1, y1 = coord1
    x2, y2 = coord2
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return dist


def generate_initial_route(city_coords):
    # Generate initial route using Randomized Nearest Neighbor algorithm
    city_indices = list(range(len(city_coords)))
    initial_route = [random.choice(city_indices)]
    city_indices.remove(initial_route[0])
    while len(city_indices) > 0:
        last_index = initial_route[-1]
        distances = [(i, distance(city_coords[i], city_coords[last_index]))
                     for i in city_indices]
        distances = sorted(distances, key=lambda x: x[1])
        nearest_index = distances[0][0]
        initial_route.append(nearest_index)
        city_indices.remove(nearest_index)
    return initial_route


def calculate_total_distance(route, dist_matrix):
    # Calculate the total distance of a route given its distance matrix
    total_distance = 0
    for i in range(len(route)-1):
        start_index = route[i]
        end_index = route[i+1]
        total_distance += dist_matrix[start_index][end_index]
    return total_distance


def generate_initial_route(city_coords, dist_matrix):
    # Generate an initial route using the randomized nearest neighbor algorithm
    num_cities = len(city_coords)
    start_city_index = random.randint(0, num_cities-1)
    unvisited_cities = set(range(num_cities))
    unvisited_cities.remove(start_city_index)
    route = [start_city_index]
    while unvisited_cities:
        nearest_city_index = min(
            unvisited_cities, key=lambda x: dist_matrix[route[-1]][x])
        unvisited_cities.remove(nearest_city_index)
        route.append(nearest_city_index)
    return route


def run_2opt(route, dist_matrix, num_iterations):
    # Improve a route using the 2-opt algorithm
    best_route = route
    best_distance = calculate_total_distance(best_route, dist_matrix)
    for i in range(num_iterations):
        new_route = best_route.copy()
        # Randomly select two cities to swap
        city1, city2 = random.sample(range(len(new_route)), 2)
        # Swap the two cities to create a new route
        new_route[city1], new_route[city2] = new_route[city2], new_route[city1]
        # Apply 2-opt to the new route
        for j in range(len(new_route)-2):
            for k in range(j+2, len(new_route)-1):
                if dist_matrix[new_route[j]][new_route[k]] + dist_matrix[new_route[j+1]][new_route[k+1]] < dist_matrix[new_route[j]][new_route[j+1]] + dist_matrix[new_route[k]][new_route[k+1]]:
                    new_route[j+1:k+1] = reversed(new_route[j+1:k+1])
        # Calculate the total distance of the new route
        new_distance = calculate_total_distance(new_route, dist_matrix)
        # If the new route is shorter, update the best route and distance
        if new_distance < best_distance:
            best_distance = new_distance
            best_route = new_route
    return best_route, best_distance


# Main code
city_coords = [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)]
dist_matrix = [[0, 1, 2, 3, 4],
               [1, 0, 1, 2, 3],
               [2, 1, 0, 1, 2],
               [3, 2, 1, 0, 1],
               [4, 3, 2, 1, 0]]
num_iterations = 500
unit = 'km'

print("City Coordinates:")
for city in city_coords:
    print(city)

print("\nDistance Matrix:")
for row in dist_matrix:
    print(row)

initial_route = generate_initial_route(city_coords, dist_matrix)
print("\nInitial Route:")
print(initial_route)
print("Initial Route Distance:", calculate_total_distance(
    initial_route, dist_matrix), unit)

best_route, best_distance = run_2opt(
    initial_route, dist_matrix, num_iterations)
print("\nBest Route:")
print(best_route)
print("Best Route Distance:", best_distance, unit)


City Coordinates:
(0, 0)
(1, 1)
(2, 2)
(3, 3)
(4, 4)

Distance Matrix:
[0, 1, 2, 3, 4]
[1, 0, 1, 2, 3]
[2, 1, 0, 1, 2]
[3, 2, 1, 0, 1]
[4, 3, 2, 1, 0]

Initial Route:
[3, 2, 1, 0, 4]
Initial Route Distance: 7 km

Best Route:
[4, 3, 2, 1, 0]
Best Route Distance: 4 km


In [55]:
# Main code
import random

# Generate 15 random cities 
# Author's (Carl's) Note: I wanted to work with the real coordinates of Manila, but I will do that in the future due to time constraints and conflicting priorities.
city_coords = [(random.uniform(-100, 100), random.uniform(-100, 100))
               for i in range(15)]

# Calculate distance matrix
dist_matrix = generate_distance_matrix(city_coords)

# Set number of iterations and unit of measurement
num_iterations = 500
unit = 'km'

print("City Coordinates:")
for city in city_coords:
    print(city)

initial_route = generate_initial_route(city_coords, dist_matrix)
print("\nInitial Route:")
print(initial_route)
print("Initial Route Distance:", calculate_total_distance(
    initial_route, dist_matrix), unit)

print("\nDistance Matrix:")
for row in dist_matrix:
    print(row)

best_route, best_distance = run_2opt(
    initial_route, dist_matrix, num_iterations)
print("\nBest Route:")
print(best_route)
print("Best Route Distance:", best_distance, unit)


City Coordinates:
(0.9286961095866815, -22.7162613164069)
(-11.986110443773995, 58.16222251621713)
(34.840557417294605, -80.68331887779536)
(-6.332683079756379, -46.86045897103168)
(-95.5491644908437, 11.83821296665279)
(50.6961237442531, -53.70704016315071)
(-2.7408802355841004, 90.15580686220363)
(68.69274857429176, 64.73151578128085)
(-4.411021251598356, 41.37512117828746)
(47.48308827251796, -72.82171839891075)
(-30.35472671908859, -73.39729871770717)
(91.83948856223455, -50.678251718867685)
(67.01935176250359, -28.377535056846057)
(-67.16932519700664, 36.32371721466174)
(52.219371824385604, -32.5244901356545)

Initial Route:
[13, 4, 1, 8, 6, 7, 12, 14, 5, 9, 2, 3, 0, 10, 11]
Initial Route Distance: 702.4218742869166 km

Distance Matrix:
[0, 81.90312189028423, 67.15797867475271, 25.212495079166864, 102.47921388757189, 58.62785367809009, 112.931703101786, 110.63037795183646, 64.31343476738398, 68.39494322760004, 59.55854343374826, 95.1138533108386, 66.33268263078634, 90.128017679422

In [41]:
# Ideal code that I didn't do because I'm a noob
MNL_cities = [
    (14.6517, 120.9842),  # Caloocan
    (14.7067, 120.9664),  # Valenzuela
    (14.6720, 120.9574),  # Malabon
    (14.6760, 121.0437),  # Quezon City
    (14.6517, 120.9842),  # Caloocan
    (14.6414, 121.0861),  # Marikina
    (14.5995, 120.9842),  # Manila
    (14.5833, 121.0500),  # Mandaluyong Pasig
    (14.5547, 121.0244),  # Makati
    (14.5378, 121.0000),  # Pasay
    (14.5176, 121.0509),  # Taguig
    (14.4793, 120.8973),  # Parañaque
    (14.3776, 120.9810),  # Las Piñas
    (14.4000, 121.0500),  # Muntinlupa
]
